In [1]:
pip install pandas numpy matplotlib statsmodels pandas_datareader pandas_ta datetime akshare scikit-learn PyPortfolioOpt

Looking in indexes: http://pypi.douban.com/simple/


Note: you may need to restart the kernel to use updated packages.


In [1]:
from statsmodels.regression.rolling import RollingOLS
import requests
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import akshare as ak
import pandas_ta
import warnings
import json
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/Users/lydiretsai/Documents/quant/AlgorithmicTradingMLQuant/stock_list.csv')
df = df.set_index('ticker') #'date', 'symbol', 'ticker']) #
df.columns.names = ['market']

In [3]:
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2 * np.log(2) - 1) * ((np.log(df['adj close']) - np.log(df['open']))) ** 2
df['rsi'] = df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
df['bb_low'] = df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 0])
df['bb_mid'] = df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 1])
df['bb_high'] = df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 2])

In [9]:
df.xs('中国重工',level=1)['rsi'].plot()

TypeError: Index must be a MultiIndex

In [5]:
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                       low=stock_data['low'],
                       close=stock_data['adj close'],
                       length=14)
    return atr.sub(atr.mean()).div(atr.std())

In [6]:
atr = df.groupby('ticker', group_keys=False).apply(compute_atr)

In [ ]:
# atr_pd = pd.DataFrame({'atr' :atr})
# df = pd.concat([df, atr])
# df.assign(new_columns=atr)
df['atr'] = df.groupby('ticker', group_keys=False).apply(compute_atr)

In [ ]:
df

In [ ]:
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:, 0]
    return macd.sub(macd.mean()).div(macd.std())

In [ ]:
macd = df.groupby('ticker', group_keys=False)['adj close'].apply(compute_macd)
macd

In [ ]:
df

In [ ]:
df